<a href="https://colab.research.google.com/github/raaz0000002/glacier_thesis/blob/main/depth_computaiton_using_contour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import pandas as pd

# Load shapefiles
lakes = gpd.read_file("/content/drive/MyDrive/Thesis/2024_inventory/updated_inventory24.shp")
contours = gpd.read_file("/content/drive/MyDrive/Thesis/2024_inventory/Conotur.shp")

# Ensure same CRS
if lakes.crs != contours.crs:
    contours = contours.to_crs(lakes.crs)

# Column containing elevation values in contours shapefile (change if named differently)
elevation_field = "Contour"  # Replace with actual elevation field name if different

# Store results
results = []

for index, lake in lakes.iterrows():
    # Intersect contours with lake polygon
    lake_gdf = gpd.GeoDataFrame([lake], geometry='geometry', crs=lakes.crs)
    intersection = gpd.overlay(lake_gdf, contours, how='intersection')

    if not intersection.empty and elevation_field in intersection.columns:
        elevations = intersection[elevation_field].astype(float)
        if len(elevations) >= 2:
            depth = elevations.max() - elevations.min()
        else:
            depth = 0  # Not enough contour lines to compute depth
    else:
        depth = 0  # No contours intersect

    results.append([lake['GLIMS_Name'], depth])  # Replace with actual lake ID column

# Merge depth data
depth_df = pd.DataFrame(results, columns=['GLIMS_Name', 'depth'])
lakes = lakes.merge(depth_df, on='GLIMS_Name', how='left')

# Save updated shapefile
lakes.to_file("/content/drive/MyDrive/Thesis/2024_inventory/lakes_with_depth.shp")

print("✅ Lake depth calculation complete. Output saved.")


<ipython-input-1-27c3d64a9969>:21: UserWarning: `keep_geom_type=True` in overlay resulted in 22819 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersection = gpd.overlay(lake_gdf, contours, how='intersection')
<ipython-input-1-27c3d64a9969>:21: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersection = gpd.overlay(lake_gdf, contours, how='intersection')
<ipython-input-1-27c3d64a9969>:21: UserWarning: `keep_geom_type=True` in overlay resulted in 9 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersection = gpd.overlay(lake_gdf, contours, how='intersection')
<ipython-input-1-27c3d64a9969>:21: UserWarning: `keep_geom_type=True` in overlay resulted in 25 dropped geometries of different geometry types than df1 has. Set `kee

✅ Lake depth calculation complete. Output saved.
